### Data visualisation

Define a visual representation for the experimental measurements of the kernel perceptron implementation, then convert them to Tikz using the `tikzplotlib` library. The only graphs needed are heatmaps and line graphs. Save the resulting .tex (and .png for the heatmaps) in `OUTPUT`.

In [1]:
import sys
sys.path.append("../")

In [2]:
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter

import numpy as np
import pandas as pd
import tikzplotlib

import os
import shutil

from KMeans import RESULTS_LOCATION, REDUCTIONS
from utils import EPOCHS, DEGREES

OUTPUT = "./output"

In [3]:
# tikzplotlib in some plots has issue rendering the labels,
# they need to be written in LateX notation

def fix(labels):
    return [f"${label}$" for label in labels]

In [4]:
def heatmap(data, attribute, reduction, max_error, min_error, save=False):
    inferno = plt.cm.get_cmap('inferno').reversed()
    error = data[attribute].to_numpy().reshape((len(EPOCHS), len(DEGREES)))[::-1]

    plt.xticks(ticks=np.arange(len(DEGREES)), labels=fix(DEGREES))
    plt.yticks(ticks=np.arange(len(EPOCHS)), labels=fix(EPOCHS[::-1]))

    hm = plt.imshow(error, cmap=inferno, vmax=max_error, vmin=min_error)
    plt.colorbar(hm, cmap=inferno, format="%0.2f")

    plt.xlabel("Grado del polinomio")
    plt.ylabel("Epoche")

    if save:
        tikzplotlib.save(f"{OUTPUT}/{reduction}_{attribute}.tex")
        plt.close()
    else:
        plt.show()

In [5]:
def lines(data, attribute, reduction, max_error, min_error, save=False):
    plt.gca().yaxis.set_major_formatter("%0.2f")

    for d in DEGREES:
        error = data[data["degree"] == d].sort_values("epochs")[attribute]
        plt.plot(EPOCHS, error, "v-", label=f"grado {d}")
    
    plt.xticks(ticks=np.arange(1, len(EPOCHS) + 1), labels=fix(EPOCHS))
    plt.xlabel("Epoche")
    
    plt.ylabel(attribute.replace("_", " ").capitalize())
    plt.legend(loc="upper right")

    if save:
        tikzplotlib.save(f"{OUTPUT}/epochs_{reduction}_{attribute}.tex")
        plt.close()
    else:
        plt.show()


    plt.gca().yaxis.set_major_formatter("%0.2f")

    for e in EPOCHS:
        error = data[data["epochs"] == e].sort_values("degree")[attribute]
        plt.plot(DEGREES, error, "v-", label=f"{e} epoche")

    plt.xticks(ticks=np.arange(1, len(DEGREES) + 1), labels=fix(DEGREES))
    plt.xlabel("Grado del polinomio")
    
    plt.ylabel(attribute.replace("_", " ").capitalize())
    plt.legend(loc="upper right")

    if save:
        tikzplotlib.save(f"{OUTPUT}/degree_{reduction}_{attribute}.tex")
        plt.close()
    else:
        plt.show()

Convert matplotlib plots into tikz plots

In [7]:
if os.path.exists(OUTPUT):
    shutil.rmtree(OUTPUT)

os.mkdir(OUTPUT)

plt.rcParams['figure.figsize'] = [10, 8]

for approach in ["min", "mean"]:
    for red in REDUCTIONS:
        max_err = pd.read_csv(f".{RESULTS_LOCATION}/{min(REDUCTIONS)}-kmmkp.csv")
        min_err = pd.read_csv(f".{RESULTS_LOCATION}/{max(REDUCTIONS)}-kmmkp.csv")

        max_test_error = max_err[f"test_error_{approach}"].max()
        min_test_error = min_err[f"test_error_{approach}"].min()

        max_training_error = max_err[f"training_error_km_{approach}"].max()
        min_training_error = min_err[f"training_error_km_{approach}"].min()

        results = pd.read_csv(f".{RESULTS_LOCATION}/{red}-kmmkp.csv")

        heatmap(results, f"test_error_{approach}", red, max_test_error, min_test_error, save=True)
        heatmap(results, f"training_error_km_{approach}", red, max_training_error, min_training_error, save=True)

        lines(results, f"test_error_{approach}", red, max_test_error, min_test_error, save=True)
        lines(results, f"training_error_km_{approach}", red, max_training_error, min_training_error, save=True)